In [1]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import os
import cv2
import torchshow as ts
from tqdm import tqdm
from dataloader.loader import generator
import yaml
import utils.utils as ut
import matplotlib.pyplot as plt
import math
ut.set_random_seed(1221)
with open('config/config.yaml', 'r') as file:
    cfg = yaml.load(file, Loader=yaml.FullLoader)

In [98]:
mask = torch.randint(0,2,(2,9,9)).float()
cfeature = torch.randn((2,256,224,224))
p_len = 8
cfeature = F.avg_pool2d(cfeature, kernel_size = p_len, stride=p_len)
Ba, Ch,_,_ = cfeature.shape
cfeature = cfeature.view(Ba, Ch, -1)
print(cfeature.shape)
cfeature = F.normalize(cfeature, dim = -1)

torch.Size([2, 256, 784])


In [6]:
(224//33)

6

In [97]:
a = torch.randn((2,5,2))
print(a)
a = torch.transpose(a,1,2)
print(a)
print(a.shape)

tensor([[[ 0.5286,  0.9433],
         [-1.1991, -1.0777],
         [-0.8163,  1.0610],
         [ 1.1719, -0.8381],
         [ 0.2265,  1.5756]],

        [[ 0.8349,  0.2023],
         [ 0.2397,  0.8597],
         [-0.6907,  1.0505],
         [ 0.1328,  0.6613],
         [ 1.1501,  0.3399]]])
tensor([[[ 0.5286, -1.1991, -0.8163,  1.1719,  0.2265],
         [ 0.9433, -1.0777,  1.0610, -0.8381,  1.5756]],

        [[ 0.8349,  0.2397, -0.6907,  0.1328,  1.1501],
         [ 0.2023,  0.8597,  1.0505,  0.6613,  0.3399]]])
torch.Size([2, 2, 5])


In [85]:
temperature = 0.6
mem_mask = torch.eq(mask, mask.transpose(1,2)).float()
mem_mask_neg = torch.add(torch.negative(mem_mask),1)


feat_logits =  torch.div(torch.matmul(feat, feat.transpose(1,2)),temperature)
identity = torch.eye(feat_logits.shape[-1])
neg_identity = torch.add(torch.negative(identity),1).detach()

feat_logits = torch.mul(feat_logits, neg_identity)

feat_logits_max, _ = torch.max(feat_logits, dim=1, keepdim=True)
feat_logits = feat_logits - feat_logits_max.detach()

feat_logits = torch.exp(feat_logits)

neg_sum = torch.sum(torch.mul(feat_logits, mem_mask_neg), dim=-1)
denominator = torch.add(feat_logits, neg_sum.unsqueeze(dim=-1))
division = torch.div(feat_logits, denominator+ 1e-18)
    
loss_matrix = -torch.log(division+1e-18)
loss_matrix = torch.mul(loss_matrix , mem_mask)
loss_matrix = torch.mul(loss_matrix, neg_identity)
loss = torch.sum(loss_matrix, dim=-1)

loss = torch.div(loss, mem_mask.sum(dim=-1) -1 + 1e-18)

In [86]:
print(loss)

tensor([[ 4.8780, 38.8191, 18.1363,  9.9295, 10.4101, 26.0503, 14.9004, 28.6080,
          9.1373],
        [ 7.9928, 20.7607, 21.2218, 21.2600, 10.7253, 36.6616, 36.6117, 30.1503,
         31.6143]])


In [68]:
denominator.shape

torch.Size([2, 9, 9])

tensor([[[0., 0., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]]])


In [9]:
from sklearn import metrics
y_true = torch.randint(0,2,(2,224,224)).numpy()
y_pred = torch.randn((2,224,224)).numpy()
auc = []
for yy_true, yy_pred in zip( y_true, y_pred ) :
    val_auc = metrics.roc_auc_score(yy_true.ravel(), yy_pred.ravel(), average = None)
    auc.append(val_auc)
print(np.mean(auc))

0.499586464589324


In [ ]:
torch.rand

In [1]:
import torch
a = torch.randn((2,2,224,224))
a = a[:,1,:,:]
print(a.shape)

torch.Size([2, 224, 224])


In [ ]:
pip install torchmetrics

In [ ]:
root_dir = '/home/agency/xai/forgery/IMD2020'
all_files = os.listdir(root_dir)
train_all_IDs = {}
mask_all_IDs = {}
count = 0
for files in all_files:
    all_file = os.listdir(os.path.join(root_dir,files))
    for file_name in all_file:
        if len(file_name.split('_0.jpg'))==2:
            train_all_IDs[count] = files+'/'+file_name
            mask_all_IDs[count] = files+'/'+file_name.split('_0.jpg')[0]+'_0_mask.png'
            count +=1
rand = np.random.choice(len(train_all_IDs), math.ceil(len(train_all_IDs)*0.1), replace=False).tolist()

train_IDs = {}
mask_IDs = {}
test_IDs = {}
mask_test_IDs = {}

count = 0
test_count = 0
for i in range(len(train_all_IDs)):
    if i in rand:
        test_IDs[test_count] = os.path.join(root_dir,train_all_IDs[i])
        mask_test_IDs[test_count] = os.path.join(root_dir,mask_all_IDs[i])
        test_count += 1
    else:
        train_IDs[count] = os.path.join(root_dir,train_all_IDs[i])
        mask_IDs[count] = os.path.join(root_dir,mask_all_IDs[i])
        count += 1